In [167]:
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split

In [168]:
def parse_data(fpath):
    data = np.genfromtxt(fpath, dtype=np.int64, skip_header=7)
    X, y = data[:, :-1], data[:, -1]
    return X, y

In [169]:
X, y = parse_data("/Users/ChesterHuynh/classes/dldo467/dldo-hw/hw2/stable5.txt")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [170]:
class My_Net(nn.Module): 
    def __init__(self, n_features, n_classes, num_neurons):
        super(My_Net, self).__init__()
        #feed forward layers
        self.layer_1 = nn.Linear(n_features, num_neurons)
        self.layer_2 = nn.Linear(num_neurons, num_neurons)
        self.layer_3 = nn.Linear(num_neurons, n_classes)

        #activations
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, input_data):
        out = self.layer_1(input_data)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.softmax(out)
        return out

In [171]:
n_classes = len(np.unique(y))
n_features = X_train.shape[1]

net=My_Net(n_features, n_classes, num_neurons=10)

# Standard cross entropy loss for multi-classification tasks
loss=nn.CrossEntropyLoss()

# Define the optimizer. Here we use Adam optimizer.
opt=torch.optim.Adam(net.parameters(),lr=1)

In [172]:
Xtrain = torch.Tensor(X_train)
Xtest = torch.Tensor(X_test)
ytrain = torch.Tensor(y_train)
ytest = torch.Tensor(y_test)

In [173]:
train = torch.utils.data.TensorDataset(Xtrain, ytrain)
test = torch.utils.data.TensorDataset(Xtest, ytest)

train_loader = torch.utils.data.DataLoader(train, batch_size=128, num_workers=2)
test_loader = torch.utils.data.DataLoader(test, batch_size=128, shuffle=False,num_workers=2)

In [174]:
def train_eval(verbose=1):
    correct = 0
    total = 0
    loss_sum = 0
    num_batches = 0
    for inputs, labels in train_loader:
        outputs = net(inputs)
        predicted = torch.argmax(outputs, dim=1) + 1
        total += labels.size(0)
        correct += (predicted.int() == labels.int()).sum()
        loss_sum  += loss(outputs,labels).item()
        num_batches += 1

    if verbose:
        print('Train accuracy: %f %%' % (100 * correct.item() / total))
    return loss_sum/num_batches, correct.item() / total

def test_eval(verbose=1):
    correct = 0
    total = 0
    loss_sum = 0
    num_batches = 0
    for inputs, labels in test_loader:
        outputs = net(inputs)
        predicted = outputs.data>0.5
        total += labels.size(0)
        correct += (predicted.int() == labels.int()).sum()
        loss_sum  += loss(outputs,labels).item()
        num_batches += 1

    if verbose:
        print('Test accuracy: %f %%' % (100 * correct.item() / total))
    return loss_sum/num_batches, correct.item() / total

In [175]:
#initialize the network using Xavier initialization.
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)

net.apply(weights_init)

My_Net(
  (layer_1): Linear(in_features=10, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=10, bias=True)
  (layer_3): Linear(in_features=10, out_features=5, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [180]:
import time

epochs=25;
train_loss_store = []
train_acc_store = []
test_loss_store = []
test_acc_store = []


for epoch in range(epochs):
    time1=time.time()
    print('In epoch %i : '%(epoch+1))
    for i, (feat, label) in enumerate(train_loader, 0):
        #set the gradients to zero initially for each batch
        opt.zero_grad()
        outputs = net(feat)
        l=loss(outputs, label)
        break
        l.backward()
        opt.step()
    break
    
    l_temp, acc_temp = train_eval()
    train_loss_store.append(l_temp)
    train_acc_store.append(acc_temp)

    l_temp, acc_temp = test_eval()
    test_loss_store.append(l_temp)
    test_acc_store.append(acc_temp)

    time2=time.time()
    print("Time lapse: %f secs" %round((time2-time1),2))

In epoch 1 : 
torch.Size([128, 5])


RuntimeError: expected scalar type Long but found Float